In [60]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt

from imutils.video import VideoStream
from imutils.video import FPS
import imutils

import math
import time

In [61]:
resW = 1920
resH = 1080
resOutH = 512

cap = cv2.VideoCapture(2)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, resW)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, resH)

cascade = cv2.CascadeClassifier('./haarcascades/barcode_0.2_2000_10.xml')
scale = 1.3
neighbors = 25
objSize = (25, 25)

font = cv2.FONT_HERSHEY_TRIPLEX
textString = ''
colorFPS = (255,255,0)
colorRect = (255, 255, 0)

capture = 0
boxExtend = 25

In [62]:
while(True):
    tick1 = cv2.getTickCount()
    (grabbed, img) = cap.read()
    if not grabbed:
        print("not grabbed")
        break
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    xmin = 9999
    xmax = 0
    ymin = 9999
    ymax = 0
    
    grayHaar = gray.copy();
    grayHaar = imutils.resize(grayHaar, height = int(resH/2))
    codes = cascade.detectMultiScale(grayHaar, scaleFactor=scale, minNeighbors=neighbors, minSize=objSize)
    
    #print(codes)
    for (x,y,w,h) in codes:
        x = x * 2;
        y = y * 2;
        w = w * 2;
        h = h * 2;
        
        if x < xmin:
            xmin = x
        if (x+w) > xmax:
            xmax = x+w
        if y < ymin:
            ymin = y
        if (y+h) > ymax:
            ymax = y+h
    
    if boxExtend != 0:
        img = cv2.rectangle(img, (xmin-boxExtend, ymin-boxExtend), (xmax+boxExtend, ymax+boxExtend), colorRect, 3)
    else:
        img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), colorRect, 3)


    imgOut = img.copy()
    tick2 = cv2.getTickCount()
    tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())
    imgOut = cv2.copyMakeBorder(imgOut, 0, 80, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    textString = str(tick) + " ms"
    cv2.putText(imgOut, textString, (10, resH+30), font, 1, colorFPS, 2, cv2.LINE_AA)
    imgOut = imutils.resize(imgOut, height = resOutH)
    cv2.imshow('output',imgOut)
    
    if capture:
        cv2.imshow('capture', img)
        fileName = "../imageCapture/" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
        cv2.imwrite(fileName, img, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        capture = 0
    
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('c'):
        capture = 1
    elif getKey is ord('q'):
        break

In [63]:
cap.release()
cv2.destroyAllWindows()